In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
compfile = pd.read_csv('delivery_orders_march.csv')

In [34]:
compfile.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_del_attempt_datetime,2nd_deliver_attempt_datetime,pick_datetime,buyeraddress_lower,selleraddress_lower
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-05 05:09:18,NaT,2020-03-02 08:39:57,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-06 02:53:56,2020-03-10 00:02:56,2020-03-04 08:19:28,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-06 01:56:19,NaT,2020-03-04 07:20:34,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-07 04:45:41,NaT,2020-03-05 14:36:56,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-06 07:41:40,NaT,2020-03-04 10:38:25,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...


Convert from epoch time to locale time.

In [7]:
type(compfile['1st_deliver_attempt'][1])

numpy.float64

In [17]:
compfile['1st_del_attempt_datetime'] = pd.to_datetime(compfile['1st_deliver_attempt'],unit='s')

In [19]:
compfile['2nd_deliver_attempt_datetime'] = pd.to_datetime(compfile['2nd_deliver_attempt'],unit='s')

In [20]:
compfile['pick_datetime'] = pd.to_datetime(compfile['pick'],unit='s')

Lowercasing the entire address columns for uniformity.

In [31]:
compfile['buyeraddress_lower'] = compfile['buyeraddress'].str.lower()

In [33]:
compfile['selleraddress_lower'] = compfile['selleraddress'].str.lower()

Running a sanity check for lengths show that some addresses contain multiple instances of different regions. We proceed by simply taking the last word in the address line as the region. Manila can be interpreted as Metro Manila, since the function only takes the last word.

In [35]:
compfile['buyeraddress_lower'].str.contains('metro manila').sum()

1560698

In [37]:
compfile['buyeraddress_lower'].str.contains('luzon').sum()

806165

In [38]:
compfile['buyeraddress_lower'].str.contains('visayas').sum()

480146

In [39]:
compfile['buyeraddress_lower'].str.contains('mindanao').sum()

331744

In [40]:
331744+480146+806165+1560698

3178753

In [36]:
len(compfile)

3176313

Some regions contain instances of multiple regions. These were used as a passing check.

In [44]:
compfile['buyeraddress_lower'].str.contains('luzon mindanao').sum()

5

In [45]:
compfile['buyeraddress_lower'].str.contains('luzon visayas').sum()

12

In [46]:
compfile['buyeraddress_lower'].str.contains('mindanao visayas').sum()

9

In [50]:
compfile['buyeraddress_lower'].str.contains('metro manila visayas').sum()

0

In [53]:
luzon_vis_mismatch = compfile[compfile['buyeraddress_lower'].str.contains('luzon visayas')]

Extracting the last word from the address columns to determine the buyer/seller region:

In [72]:
compfile['buyeraddress_city'] = compfile['buyeraddress_lower'].str.split().str[-1]

In [75]:
compfile['selleraddress_city'] = compfile['selleraddress_lower'].str.split().str[-1]

Checking the locations of all the buyers to see if the data makes sense:

In [77]:
compfile['buyeraddress_city'].value_counts()

manila      1560698
luzon        804888
visayas      479595
mindanao     331132
Name: buyeraddress_city, dtype: int64

Buyer addresses look good. No surprises here.

Nearly all sellers are located in Manila. Key thing to look out for:

In [76]:
compfile['selleraddress_city'].value_counts()

manila    3176312
luzon           1
Name: selleraddress_city, dtype: int64

In [78]:
compfile.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_del_attempt_datetime,2nd_deliver_attempt_datetime,pick_datetime,buyeraddress_lower,selleraddress_lower,buyeraddress_city,selleraddress_city
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-05 05:09:18,NaT,2020-03-02 08:39:57,"baging ldl buenavista,patag.cagayan buagsong,c...",pantranco vill. 417 warehouse# katipunan 532 (...,manila,manila
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-06 02:53:56,2020-03-10 00:02:56,2020-03-04 08:19:28,coloma's quzom casanas site1 masiyan 533a stol...,"bldg 210a moras c42b 2b16,168 church) complex ...",manila,manila
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",2020-03-06 01:56:19,NaT,2020-03-04 07:20:34,"21-o lumangdaan,capitangan,abucay,bataan .bign...","#66 150-c, drive, milagros joe socorro metro m...",manila,manila
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",2020-03-07 04:45:41,NaT,2020-03-05 14:36:56,"616espiritu martinville,manuyo #5paraiso kengi...","999maii 201,26 villaruel barretto gen.t number...",manila,manila
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,2020-03-06 07:41:40,NaT,2020-03-04 10:38:25,l042 summerbreezee1 l2(balanay analyn lot760 c...,g66manila hiyas fitness maysilo magdiwang lt.4...,luzon,manila


Find number of days between pickup and 1st delivery:

In [90]:
sample_order = compfile.iloc[1]
sample_order

orderid                                                                2219624609
pick                                                                   1583309968
1st_deliver_attempt                                                   1.58346e+09
2nd_deliver_attempt                                                    1.5838e+09
buyeraddress                    coloma's quzom CASANAS Site1 Masiyan 533A Stol...
selleraddress                   BLDG 210A Moras C42B 2B16,168 church) Complex ...
1st_del_attempt_datetime                                      2020-03-06 02:53:56
2nd_deliver_attempt_datetime                                  2020-03-10 00:02:56
pick_datetime                                                 2020-03-04 08:19:28
buyeraddress_lower              coloma's quzom casanas site1 masiyan 533a stol...
selleraddress_lower             bldg 210a moras c42b 2b16,168 church) complex ...
buyeraddress_city                                                          manila
selleraddress_ci

In [85]:
sample_order['1st_del_attempt_datetime'].weekday()

4

In [86]:
sample_order['pick_datetime'].weekday()

2

In [87]:
sample_order['2nd_deliver_attempt_datetime'].weekday()

1

In [105]:
np.busday_count(sample_order['pick_datetime'].date(), sample_order['1st_del_attempt_datetime'].date(), weekmask=[1,1,1,1,1,1,0], holidays=holiday_list) - 1

1


Find orders that fall in public holiday range. 

In [103]:
holiday_list = ['2020-03-08','2020-03-25', '2020-03-30', '2020-03-31']

Converted the datetime columns into standalone date columns for the np method to ingest

In [120]:
compfile['pick_date'] = pd.to_datetime(compfile['pick_datetime']).dt.date  
compfile['1st_del_date'] = pd.to_datetime(compfile['1st_del_attempt_datetime']).dt.date  


KeyError: '2nd_del_attempt_datetime'

In [123]:
compfile['2nd_del_date'] = pd.to_datetime(compfile['2nd_deliver_attempt_datetime']).dt.date  


The workhorse function, np.busday_count which takes in both eligible weekdays as well as holidays. Almost perfect for this problem.

In [232]:
compfile['pickup_1st_del_range'] = np.busday_count(compfile['pick_date'], compfile['1st_del_date'], weekmask=[1,1,1,1,1,1,0], holidays=holiday_list)

Secondary df to deal with the NaN values. On hindsight, should have used a function to replace NaT values with the 1st_del_date so that the difference would equal 0.

In [233]:
temp_2nd_del_df['1st_del_2nd_del_range'] = np.busday_count(temp_2nd_del_df['1st_del_date'], temp_2nd_del_df['2nd_del_date'], weekmask=[1,1,1,1,1,1,0], holidays=holiday_list)

/Users/arnybarnie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [166]:
temp_2nd_del_df['1st_del_2nd_del_range'] = temp_2nd_del_df['1st_del_2nd_del_range'].astype('int64')

/Users/arnybarnie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [234]:
compfile_processed_df = pd.merge(compfile, temp_2nd_del_df[['orderid', '1st_del_2nd_del_range']], on='orderid', how='left')

In [176]:
compfile_processed_df['1st_del_2nd_del_range'].astype('int64')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [157]:
sample_order_2 = compfile_processed_df.iloc[:2]

In [201]:
sample_order_2['pick_1st_range'] = ((sample_order_2['selleraddress_city'] == 'manila') & (sample_order_2['buyeraddress_city']=='manila') & (sample_order_2['pickup_1st_del_range'] <= 3 ) | 
                                    (sample_order_2['selleraddress_city'] == 'manila') & (sample_order_2['buyeraddress_city']=='luzon') & (sample_order_2['pickup_1st_del_range'] <= 5 ) |
                                    (sample_order_2['selleraddress_city'] == 'manila') & (sample_order_2['buyeraddress_city']=='visayas') & (sample_order_2['pickup_1st_del_range'] <= 7 ) |
                                    (sample_order_2['selleraddress_city'] == 'manila') & (sample_order_2['buyeraddress_city']=='mindanao') & (sample_order_2['pickup_1st_del_range'] <= 7 ) |
                                    (sample_order_2['selleraddress_city'] == 'luzon') & (sample_order_2['buyeraddress_city']=='luzon') & (sample_order_2['pickup_1st_del_range'] <= 5 ) 
                                   
                                   )

In [207]:
sample_order_2['1st_2nd_range'] = ((sample_order_2['1st_del_2nd_del_range'].notnull()) & 
                                   (sample_order_2['1st_del_2nd_del_range'] <= 3)
                                  ) 

In [218]:
sample_order_2.loc[(sample_order_2['pick_1st_range'] == True) & (sample_order_2['1st_2nd_range'] == False) , 'final_outcome'] = 0
sample_order_2.loc[(sample_order_2['pick_1st_range'] == True) & (sample_order_2['1st_2nd_range'] == True) , 'final_outcome'] = 0
sample_order_2.loc[(sample_order_2['pick_1st_range'] == False) & (sample_order_2['1st_2nd_range'] == False) , 'final_outcome'] = 1 
sample_order_2.loc[(sample_order_2['pick_1st_range'] == False) & (sample_order_2['1st_2nd_range'] == True) , 'final_outcome'] = 1


Doing it on the real dataset:

In [235]:
compfile_processed_df['pick_1st_range'] = ((compfile_processed_df['selleraddress_city'] == 'manila') & (compfile_processed_df['buyeraddress_city']=='manila') & (compfile_processed_df['pickup_1st_del_range'] <= 3 ) | 
                                    (compfile_processed_df['selleraddress_city'] == 'manila') & (compfile_processed_df['buyeraddress_city']=='luzon') & (compfile_processed_df['pickup_1st_del_range'] <= 5 ) |
                                    (compfile_processed_df['selleraddress_city'] == 'manila') & (compfile_processed_df['buyeraddress_city']=='visayas') & (compfile_processed_df['pickup_1st_del_range'] <= 7 ) |
                                    (compfile_processed_df['selleraddress_city'] == 'manila') & (compfile_processed_df['buyeraddress_city']=='mindanao') & (compfile_processed_df['pickup_1st_del_range'] <= 7 ) |
                                    (compfile_processed_df['selleraddress_city'] == 'luzon') & (compfile_processed_df['buyeraddress_city']=='luzon') & (compfile_processed_df['pickup_1st_del_range'] <= 5 ) 
                                   #True if on time
                                   )

In [236]:
compfile_processed_df['1st_2nd_range'] = ((compfile_processed_df['1st_del_2nd_del_range'].notnull()) & 
                                   (compfile_processed_df['1st_del_2nd_del_range'] > 3)
                                  ) #true if late

In [237]:
compfile_processed_df.loc[(compfile_processed_df['pick_1st_range'] == True) & (compfile_processed_df['1st_2nd_range'] == False) , 'is_late'] = 0 #1st del: ontime, 2nd:ontime
compfile_processed_df.loc[(compfile_processed_df['pick_1st_range'] == True) & (compfile_processed_df['1st_2nd_range'] == True) , 'is_late'] = 1 #1st del on time, 2nd:late
compfile_processed_df.loc[(compfile_processed_df['pick_1st_range'] == False) & (compfile_processed_df['1st_2nd_range'] == False) , 'is_late'] = 1 #1st del late, 2nd:ontime
compfile_processed_df.loc[(compfile_processed_df['pick_1st_range'] == False) & (compfile_processed_df['1st_2nd_range'] == True) , 'is_late'] = 1 #1st del late, 2nd:late


In [238]:
compfile_processed_df['is_late'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: is_late, dtype: float64

In [239]:
compfile_processed_df['is_late'] = compfile_processed_df['is_late'].astype('int')

In [240]:
header = ["orderid", 'is_late']
compfile_processed_df.to_csv('comp5_submission_4.csv', columns = header, index=False)

In [219]:
sample_order_2

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,1st_del_attempt_datetime,2nd_deliver_attempt_datetime,pick_datetime,buyeraddress_lower,...,buyeraddress_city,selleraddress_city,pick_date,1st_del_date,2nd_del_date,pickup_1st_del_range,1st_del_2nd_del_range,pick_1st_range,1st_2nd_range,final_outcome
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,2020-03-05 05:09:18,NaT,2020-03-02 08:39:57,"baging ldl buenavista,patag.cagayan buagsong,c...",...,manila,manila,2020-03-02,2020-03-05,NaT,2,NaN,True,False,0.0
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",2020-03-06 02:53:56,2020-03-10 00:02:56,2020-03-04 08:19:28,coloma's quzom casanas site1 masiyan 533a stol...,...,manila,manila,2020-03-04,2020-03-06,2020-03-10,1,2.0,True,True,0.0
